<a href="https://colab.research.google.com/github/jasonnoy/COMP5329NEW/blob/jjh/5329assn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive to colabif not mounted before
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
# import necessary library
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from torch import nn
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import datasets
from torchvision import transforms

from sklearn.metrics import f1_score
from PIL import Image

DEBUG = False

In [ ]:
# download data from kaggle server with token file
# must update before use in Colab!
!pip install --upgrade --force-reinstall --no-deps kaggle

# following method of download dataset is from kaggle official forum : https://www.kaggle.com/general/74235
from google.colab import files
print("Please upload your kaggle.jasn token file, to allow direct file download from kaggle to Colab")
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c '2021s1comp5329assignment2'
! unzip -q 2021s1comp5329assignment2.zip
! rm 2021s1comp5329assignment2.zip

data_root_dir = "/content/COMP5329S1A2Dataset/data/"
print("DONE! image files in : ",data_root_dir)

In [ ]:
print("Please upload fixed train.csv and test.csv")
from google.colab import files
files.upload()
train_csv=pd.read_csv("train.csv")
label_column=train_csv['Labels']
img_column=train_csv['ImageID']
print(label_column)

In [ ]:
# create colums for each class
col_list=np.arange(20)
print(col_list)

# create zeros in np with number of sample from train.csv
labels=pd.DataFrame(np.zeros((len(train_csv),20), dtype=np.int), columns=col_list)
print("zeros shape :",labels.shape)

# change lables matrix value for all samples with split
index=0
for row in label_column:
    label_list=list(map(int, row.split(" ")))
    for i in label_list:
        labels.iloc[index,i] = 1
    index+=1
print(labels)

# combine image file name and label matrix
# and store this dataframe to file
processed_train_set = pd.concat([img_column, labels], axis=1)
print(processed_train_set)
processed_train_set.to_csv("processed_train.csv",index=False)

In [ ]:
# class distribution
class_sum = labels.sum(axis=0)
class_percentage = class_sum/class_sum.sum(axis=0)
for it in col_list:
    print("%3d" %it,end='\t')
print()
for it in col_list:
    print("%.3f" %class_percentage[it],end='\t')

# plot class distribution
fig = plt.figure(figsize =(10, 6))
plt.bar(col_list, class_percentage)
plt.xticks(col_list, col_list)
plt.show()

# image dataset statistics
image_infos = []
tran_stat = transforms.Compose([transforms.ToTensor()])
for it in processed_train_set.ImageID:
    image = np.array(cv2.cvtColor(cv2.imread(data_root_dir+it), cv2.COLOR_BGR2RGB))
    curr_infos = []
    # image N rows and N cols
    curr_infos.append(image.shape[0])
    curr_infos.append(image.shape[1])
    image = tran_stat(image)
    # image RGB value means
    curr_infos.append(image[:,:,0].mean())
    curr_infos.append(image[:,:,1].mean())
    curr_infos.append(image[:,:,2].mean())
    # image RGB value stds
    curr_infos.append(np.sqrt(image[:,:,0].var()+1e-5))
    curr_infos.append(np.sqrt(image[:,:,1].var()+1e-5))
    curr_infos.append(np.sqrt(image[:,:,2].var()+1e-5))
    image_infos.append(curr_infos)

image_infos = np.array(image_infos)
print("\t row \t col \t RGB means \t RGB stds")
image_infos = image_infos.mean(axis=0)
print(image_infos)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_infos, data_root_dir=data_root_dir, csv_name="processed_train.csv"):
        self.image_infos = image_infos
        print(self.image_infos)
        self.data_root_dir = data_root_dir
        self.csv_file_label = pd.read_csv(csv_name)
        self.transforms = transforms.Compose([transforms.Resize((int(self.image_infos[0]),int(self.image_infos[1]))) , 
                                            transforms.ToTensor(),
                                            transforms.Normalize([self.image_infos[2], self.image_infos[3], self.image_infos[4]], 
                                                                 [self.image_infos[5], self.image_infos[6], self.image_infos[7]])])
        #self.transforms = transforms.Compose([transforms.Resize((244,244)) , 
        #                       transforms.ToTensor(),
        #                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #                       ])
    def __len__(self):
        return len(self.csv_file_label)
    
    def __getitem__(self, index):
        #image = cv2.cvtColor(cv2.imread(self.data_root_dir + self.csv_file_label.iloc[index].ImageID), cv2.COLOR_BGR2RGB)
        image = Image.open(self.data_root_dir + self.csv_file_label.iloc[index].ImageID).convert("RGB")
        label = torch.tensor(self.csv_file_label.iloc[index][1:].tolist(), dtype=torch.float32)
        
        return self.transforms(image), label

class ImageDataloader():
    def __init__(self, image_infos, data_root_dir):
        train_percentage = 0.85
        image_dataset = ImageDataset(image_infos, data_root_dir, "processed_train.csv")
        dataset_len = len(image_dataset)
        self.train_set, self.valid_set = random_split(image_dataset, [int(dataset_len*train_percentage), (dataset_len-int(dataset_len*train_percentage))])

    def make_loader(self, batch_size):
        return DataLoader(self.train_set, shuffle=True, batch_size=batch_size), DataLoader(self.valid_set, shuffle=True, batch_size=batch_size)




In [ ]:
import torch.nn as nn
class AsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8, disable_torch_grad_focal_loss=True):
        super(AsymmetricLoss, self).__init__()

        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
        self.eps = eps

    def forward(self, x, y):
        """"
        Parameters
        ----------
        x: input logits
        y: targets (multi-label binarized vector)
        """

        # Calculating Probabilities
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE calculation
        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)  # pt = p if t > 0 else 1-p
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            loss *= one_sided_w

        return -loss.sum()

In [ ]:
class resnet_50:
    def __init__(self, N_classes=20, dropout_p=0.5, N_fc_layers=3):
        """
        :type dropout_p: float
        :param: dropout_p: the chance of nodes stay in the network
        """
        #check device, if GPU avaliable for torch
        self.device = torch.device("cpu")
        if torch.cuda.is_available():
            self.device = torch.device("cuda")

        self.model = models.resnet50(pretrained=True)
        self.N_classes = N_classes
        # decide is need dropout, base on dropout_p value
        # if dropout_p == 1, all node preserved, so no dropout
        self.dropout_p = dropout_p
        self.dropout = False
        if self.dropout_p != 1.0:
            self.dropout = True

        for param in self.model.parameters():
            param.requires_grad = False

        # connect Resnet50 to N_classes output fc
        self.activ_func = nn.ReLU
        N_fc_in = self.model.fc.in_features
        self.N_fc_layers = N_fc_layers
        fc_layers = []

        if self.dropout:
            for N_it in range(self.N_fc_layers):
                fc_layers.append(nn.Linear(N_fc_in//(2**N_it), N_fc_in//(2**(N_it+1))))
                fc_layers.append(nn.ReLU())
                fc_layers.append(nn.BatchNorm1d(N_fc_in//(2**(N_it+1))))
                # nn.dropout use probability of dropout as input
                fc_layers.append(nn.Dropout(1-self.dropout_p))
        else:
            for N_it in range(self.N_fc_layers):
                fc_layers.append(nn.Linear(N_fc_in//(2**N_it), N_fc_in//(2**(N_it+1))))
                fc_layers.append(nn.ReLU())
                fc_layers.append(nn.BatchNorm1d(N_fc_in//(2**(N_it+1))))
        
        # last layer to connect to N_classes
        fc_layers.append(nn.Linear(N_fc_in//(2**(self.N_fc_layers)), self.N_classes))
        # add custom fc layers to Resnet
        self.model.fc = nn.Sequential(*fc_layers)

        if DEBUG:
            print(self.model)

        # change device
        self.model = self.model.to(self.device)

        #define the min threshold for prediction
        self.predict_threshold = 0.5

    def accuracy_calculate(self, y_true, outputs_torch, curr_batch_size):
        y_true = y_true.to(torch.int).numpy()
        y_pred = (torch.sigmoid(outputs_torch).data > self.predict_threshold).cpu().detach().to(torch.int).numpy()
        return f1_score(y_true, y_pred, average='samples')*curr_batch_size

    def train(self, train_dataloader, valid_dataloader, N_epochs=10, learning_rate=0.01, optimizer="adam", scheduler="steplr"):
        self.learning_rate = learning_rate
        self.len_train = len(train_dataloader.dataset)
        self.len_valid = len(valid_dataloader.dataset)

        self.optimizer = None
        # add training optimizers
        if optimizer == "adamw":
            print("optimizer : AdamW")
            self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        elif optimizer == "sparseadam":
            print("optimizer : SparseAdam")
            self.optimizer = torch.optim.SparseAdam(self.model.parameters(), lr=self.learning_rate)
        else:
            print("optimizer : Adam")
            # default as Adam
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # add training schedulers
        self.scheduler = None
        if scheduler == "reducelr":
            print("scheduler : ReduceLROnPlateau")
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')
        elif scheduler == "cosinelr":
            print("scheduler : CosineAnnealingLR")
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=5, eta_min=0.005)
        else:
            print("scheduler : StepLR")
            # default as steplr
            self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=5, gamma=0.1)

        # add training optimizaers
        self.criterion = nn.BCEWithLogitsLoss()
        #self.criterion = AsymmetricLoss()

        # recode evaluation matries for each epoch
        #============================train==============================
        for epoch_it in range(N_epochs):
            #epoch loop
            print("\nepoch:", epoch_it, '/', N_epochs,end=' ')
            train_accuracy = 0.0 
            train_loss = 0.0
            self.model.train()

            for images, labels in train_dataloader:
                torch.set_grad_enabled(True)
                # batch loop
                images = images.to(self.device)
                labels_processed = labels.to(self.device)
                curr_batch_size = images.size(0)
                
                # get output and calculate accuracy
                outputs = self.model(images)
                train_accuracy += self.accuracy_calculate(labels, outputs, curr_batch_size)

                # loss & backpropergate
                loss = self.criterion(outputs, labels_processed)
                loss.backward()
                train_loss += curr_batch_size*loss.item()

                # update parameters by grads & reset grads
                self.optimizer.step()
                self.optimizer.zero_grad()
            
            # train print out
            print("train loss:", (train_loss/self.len_train),
                  "accuracy:", (train_accuracy/self.len_train),end=' ')

            #============================valid==============================
            valid_accuracy = 0.0
            valid_loss = 0.0
            self.model.eval()
       
            for images, labels in valid_dataloader:
                torch.set_grad_enabled(False)
                # batch loop
                images = images.to(self.device)
                labels_processed = labels.to(self.device)
                curr_batch_size = images.size(0)
                
                # get output and calculate accuracy
                outputs = self.model(images)
                valid_accuracy += self.accuracy_calculate(labels, outputs, curr_batch_size)

                # loss & backpropergate
                loss = self.criterion(outputs, labels_processed)
                valid_loss += curr_batch_size*loss.item()

            # valid print out
            print("\tvalid loss:", (valid_loss/self.len_valid),
                  "accuracy", (valid_accuracy/self.len_valid),end='')
            
            self.scheduler.step()   
      
      def save(name):
        torch.save(model, "my_model_"+name+".pth")
      


In [ ]:
DEBUG = False

for dropout_p in [0.8]:
    print("\n===============dropout_p:",dropout_p)
    i=1
    for optimizer in ["adam", "adamw"]:
        res50 = resnet_50(N_classes=20, dropout_p=dropout_p, N_fc_layers=3)

        dataset_loader = ImageDataloader(image_infos, data_root_dir)
        (train_set, valid_set) = dataset_loader.make_loader(batch_size = 128)
        res50.train(train_dataloader=train_set, valid_dataloader=valid_set, N_epochs=20, learning_rate=0.001, optimizer=optimizer, scheduler="cosinelr")
        res50.save(i)
        i+=1

In [ ]:
def load_model(file):
  return pytorch.load(file)